In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
# OLS
import statsmodels.api as sm
# Moran's I
from libpysal.weights import KNN
from esda.moran import Moran
# Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# !pip install esda

In [2]:
hdb = gpd.read_parquet("processed_n/hdb_ols.parquet")
hdb.head(2)

,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,resale_year,resale_age,...,model_Model A2,model_Multi Generation,model_New Generation,model_Premium Apartment,model_Premium Apartment Loft,model_Simplified,model_Standard,model_Terrace,model_Type S1,model_Type S2
0,2023-01,ANG MO KIO,2 ROOM,01 TO 03,44.0,Improved,1979,267000.0,2023,44,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2023-01,ANG MO KIO,2 ROOM,04 TO 06,49.0,Improved,1977,300000.0,2023,46,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(hdb['dist_mrt'])

In [ ]:
num_vars = ['floor_area_sqm', 'resale_age', 'storey_mid',
            'dist_mrt', 'dist_hcen', 'dist_scen', 'bus_count_400m']
cat_vars = [c for c in hdb.columns if c.startswith('type_') or c.startswith('model_')]

X = hdb[num_vars + cat_vars]
y = hdb['log_price']

In [3]:
# wittout hdb type variables etc.
num_vars = ['floor_area_sqm', 'resale_age', 'storey_mid',
            'dist_mrt', 'dist_hcen', 'dist_scen', 'bus_count_400m']
# cat_vars = [c for c in hdb.columns if c.startswith('type_') or c.startswith('model_')]

X = hdb[num_vars]
y = hdb['log_price']

In [4]:
# Covert distances to km
for col in ['dist_mrt', 'dist_hcen', 'dist_scen']:
    X[col] = X[col] / 1000
X.head(2)

C:\Users\METAYJ\AppData\Local\Temp\ipykernel_46532\3036019829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col] / 1000


,floor_area_sqm,resale_age,storey_mid,dist_mrt,dist_hcen,dist_scen,bus_count_400m
0,44.0,44,2.0,0.934249,0.148675,0.483225,10
1,49.0,46,5.0,0.248135,0.747312,0.570211,10


In [5]:
# add constant
"""
add an intercept term (constant term) to your predictor variables (X)
$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + .... + \epsilon$
where: \beta_0 is the intercept (constant term); \beta_1_2_3_... are coefficients for features x_1, x_2,...
\epsilon is the error term

Without the intercept, the regression line is forced to pass through the origin (0,0), which often leads to biased models.
"""
X = sm.add_constant(X)

In [6]:
X.head(1)

,const,floor_area_sqm,resale_age,storey_mid,dist_mrt,dist_hcen,dist_scen,bus_count_400m
0,1.0,44.0,44,2.0,0.934249,0.148675,0.483225,10


In [7]:
# fit global OLS model
ols = sm.OLS(y, X).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:              log_price   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                 1.181e+04
Date:                Wed, 12 Nov 2025   Prob (F-statistic):               0.00
Time:                        16:45:23   Log-Likelihood:                 13286.
No. Observations:               25760   AIC:                        -2.656e+04
Df Residuals:                   25752   BIC:                        -2.649e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             12.5032      0.006   2095.

In [8]:
# new add VIF
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assume X already includes all predictors (with constant)
# and that model was built as: model = sm.OLS(y, X).fit()

# Drop the constant column for VIF calculation (to avoid redundancy)
X_vif = X.drop(columns='const', errors='ignore')

# Compute VIF for each column
vif_data = pd.DataFrame()
vif_data['Variable'] = X_vif.columns
vif_data['VIF'] = [variance_inflation_factor(X_vif.values, i)
                   for i in range(X_vif.shape[1])]

# Sort by descending VIF for clarity
vif_data.sort_values('VIF', ascending=False, inplace=True)
print(vif_data)

         Variable        VIF
0  floor_area_sqm  12.300084
6  bus_count_400m   8.032093
5       dist_scen   4.389126
4       dist_hcen   4.183524
1      resale_age   3.412582
3        dist_mrt   3.258993
2      storey_mid   3.003985


In [ ]:
# Test residual spatial dependence (Moran's I)
"""
If I>0 and p<0.05, spatial clustering in residuals and use GWR next.
If not significant, OLS suffices.
"""
w = KNN.from_dataframe(hdb, k=8)
w.transform = 'R'
mi = Moran(ols.resid, w)
print(f"Moran's I: {mi.I: .3f}, p-value: {mi.p_sim: .4f}")

In [ ]:
# Random Forest benchmark
"""
If RF >> OLS, it signals strong non-linearity, though OLS remains your interpretable baseline
"""
rf = RandomForestRegressor(n_estimators=500, random_state=42)
rf.fit(X.drop(columns='const'), y)
pred = rf.predict(X.drop(columns='const'))
print("RF R^2: ", r2_score(y, pred))

In [ ]:
# Save diagnostics and plots
pd.DataFrame({'coef': ols.params, 'pval': ols.pvalues}).to_csv("ols_results.csv")

# residuals map
hdb['resid'] = ols.resid
hdb.plot(column='resid', cmap='coolwarm', legend=True, figsize=(7,7))
plt.title("OLS Residuals (log_price)")
plt.show()

In [ ]:
PLAN_PATH = "processed/mp19_subzones_3414.geojson"
pla = gpd.read_file(PLAN_PATH)
pla.plot()

In [ ]:
# Symmetric color range around 0 (better for residuals)
rmax = np.nanmax(np.abs(hdb['resid'].values))
vmin, vmax = -rmax, rmax

# plot both on one map
fig, ax = plt.subplots(figsize=(7,7))

# base choropleth of residuals
hdb.plot(
    ax = ax,
    column = 'resid',
    cmap = 'coolwarm',
    vmin=vmin,
    vmax=vmax,
    linewidth=0,
    edgecolor='none',
    legend=True,
    legend_kwds=dict(label="OLS residuals (log_price)", shrink=0.75)
)

# overlay planning subzone boundaries
pla.boundary.plot(ax=ax, linewidth=0.6, color='black', alpha=0.8)

# Styling
ax.set_title("OLS Residuals (log_price)", pad=10)
ax.set_axis_off()
plt.tight_layout()
# plt.savefig("output/residuals_with_subzones.png", bbox_inches='tight', dpi=300)
plt.show()